# Projet SIEM : Suricata, Filebeat, Elasticsearch

## Objectifs

Ce projet vous permettra de :
- Vérifier le bon fonctionnement de la stack SIEM
- Analyser les données indexées par Filebeat (Suricata)
- Détecter des anomalies sans ML
- Détecter des anomalies avec ML (Isolation Forest)

## Modalité
- groupe de 3 à 4
- output : projet git avec ce notebook détaillé et complété

## Architecture SIEM

```
Suricata   →   Filebeat   →   Elasticsearch   →   Kibana
(IDS/HIDS)   (Collecteur)      (Stockage)   (Visualisation/Alerting)
```

## Prérequis

1. Démarrer la stack :
```bash
docker-compose -f docker-compose-siem.yml up -d
```

2. Attendre quelques minutes que Suricata génère des logs et que Filebeat les indexe dans Elasticsearch.

In [8]:
# Configuration et connexion à Elasticsearch
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta
import json
import subprocess
import os
import warnings
from urllib3.exceptions import InsecureRequestWarning

warnings.simplefilter("ignore", InsecureRequestWarning)
# Configuration
ES_HOST = "https://localhost:9200"
ES_USER = "elastic"
ES_PASSWORD = "changeme"  # Modifiez selon votre .env

# Connexion
es = Elasticsearch(
    [ES_HOST],
    basic_auth=(ES_USER, ES_PASSWORD),
    verify_certs=False
)

# Vérification de la connexion
health = es.cluster.health()
print(f"✅ Cluster Elasticsearch: {health['status']} ({health['number_of_nodes']} nœuds)")

✅ Cluster Elasticsearch: yellow (1 nœuds)


/home/mel/Documents/JULIE-DATA/projet_data_bis/elasticsearch_cookbook/.venv/lib/python3.12/site-packages/elasticsearch/_sync/client/__init__.py:313: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


## 1. Vérification de la stack

In [2]:
# Vérification des services Docker
services = ['es01', 'es02', 'es03', 'kibana', 'suricata', 'filebeat']
running = []

for service in services:
    try:
        result = subprocess.run(
            ['docker', 'ps', '--filter', f'name={service}', '--format', '{{.Names}}'],
            capture_output=True, text=True, timeout=5
        )
        if service in result.stdout:
            running.append(service)
            print(f"✅ {service}")
        else:
            print(f"❌ {service}")
    except:
        print(f"❌ {service}")

if len(running) == len(services):
    print(f"\n✅ Tous les services sont démarrés ({len(running)}/{len(services)})")
else:
    print(f"\n⚠️  Services démarrés: {len(running)}/{len(services)}")

❌ es01
❌ es02
❌ es03
❌ kibana
❌ suricata
❌ filebeat

⚠️  Services démarrés: 0/6


## 2. Vérification de l'injection des données

In [3]:
# Recherche des index Suricata
def get_suricata_index():
    """Retourne le nom de l'index Suricata le plus récent"""
    try:
        indices = es.indices.get(index="suricata-*")
        if indices:
            return sorted(indices.keys())[-1]
    except:
        pass
    return "suricata-*"

index_name = get_suricata_index()

# Comptage des documents
try:
    count = es.count(index=index_name)
    print(f"📊 Index: {index_name}")
    print(f"📈 Nombre de documents: {count['count']:,}")
    
    # Exemple de document
    if count['count'] > 0:
        sample = es.search(index=index_name, size=1, query={"match_all": {}})
        if sample['hits']['hits']:
            doc = sample['hits']['hits'][0]['_source']
            print(f"\n📄 Exemple de document:")
            print(f"   Type: {doc.get('event_type', 'N/A')}")
            print(f"   Timestamp: {doc.get('@timestamp', doc.get('timestamp', 'N/A'))}")
            if 'src_ip' in doc:
                print(f"   Source: {doc.get('src_ip')}:{doc.get('src_port', 'N/A')}")
                print(f"   Destination: {doc.get('dest_ip')}:{doc.get('dest_port', 'N/A')}")
            if 'alert' in doc:
                alert = doc['alert']
                print(f"   Alerte: {alert.get('signature', 'N/A')}")
                print(f"   Sévérité: {alert.get('severity', 'N/A')}")
except Exception as e:
    print(f"❌ Erreur: {e}")



❌ Erreur: name 'es' is not defined


## 3. Simuler des comportements anormaux


#### Installation des règles de détections

In [64]:
import subprocess
import time
from pathlib import Path

print("📝 Création des règles de détection spécifiques...\n")

rules_content = """# Test basique
alert http any any -> any any (msg:"HTTP Traffic"; sid:1000001; rev:1;)

# SQL Injection
alert http any any -> any any (msg:"SQL Injection"; content:"' OR '1'='1"; nocase; sid:1000002; rev:1;)

# XSS
alert http any any -> any any (msg:"XSS Attack"; content:"<script>"; nocase; sid:1000003; rev:1;)

# User-Agent suspect
alert http any any -> any any (msg:"SQLMap Tool"; content:"User-Agent: sqlmap"; nocase; sid:1000004; rev:1;)

# Test IDS
alert http any any -> any any (msg:"IDS Test"; content:"uid=0|28|root|29|"; sid:2100498; rev:7;)

# Port Scan
alert tcp any any -> any any (msg:"PORT SCAN DETECTED"; flags:S; threshold:type both, track by_src, count 10, seconds 10; sid:5000001; rev:1;)

# HTTP Burst
alert http any any -> any any (msg:"HTTP BURST DETECTED"; threshold:type both, track by_src, count 15, seconds 2; sid:5000002; rev:1;)

# Brute Force
alert http any any -> any any (msg:"BRUTE FORCE DETECTED"; content:"pass="; http_uri; threshold:type both, track by_src, count 6, seconds 10; sid:5000003; rev:1;)
"""

# 1️⃣ Créer fichier local
local_path = Path("suricata.rules")
local_path.write_text(rules_content)

# 2️⃣ Copier dans le container
subprocess.run("docker cp suricata.rules suricata:/var/lib/suricata/rules/suricata.rules", shell=True)

print("✓ Fichier copié dans le container")

# 3️⃣ Vérifier
result = subprocess.run(
    "docker exec suricata cat /var/lib/suricata/rules/suricata.rules",
    shell=True,
    capture_output=True,
    text=True
)
print(f"✓ Contenu vérifié : {len(result.stdout)} caractères\n")

# 4️⃣ Ajouter dans suricata.yaml si nécessaire
check = subprocess.run(
    'docker exec suricata grep -q "suricata.rules" /etc/suricata/suricata.yaml',
    shell=True
)

if check.returncode != 0:
    print("Ajout de suricata.rules dans suricata.yaml...")
    subprocess.run(
        'docker exec suricata sed -i "/rule-files:/a\\  - suricata.rules" /etc/suricata/suricata.yaml',
        shell=True
    )
    print("✓ suricata.rules ajouté à suricata.yaml")
else:
    print("✓ suricata.rules déjà présent")

# 5️⃣ Redémarrage
print("🔄 Redémarrage de Suricata...")
subprocess.run("docker-compose restart suricata", shell=True)

print("⏳ Attente du redémarrage (8 secondes)...")
time.sleep(8)

print("✅ Configuration terminée !")

📝 Création des règles de détection spécifiques...

✓ Fichier copié dans le container
✓ Contenu vérifié : 1036 caractères

✓ suricata.rules déjà présent
🔄 Redémarrage de Suricata...


no such service: suricata


⏳ Attente du redémarrage (8 secondes)...
✅ Configuration terminée !


#### Simulation de scans suspects (en ligne de commande ou en python) 

In [78]:
"""
═══════════════════════════════════════════════════════
ATTAQUE - PORT SCAN
═══════════════════════════════════════════════════════
Envoie 12 paquets TCP SYN en moins de 10 secondes
"""

import subprocess
import time

print("🔍 ATTAQUE 1 : PORT SCAN")
print("=" * 60)
print("\n📋 Objectif : Déclencher 'PORT SCAN DETECTED'")
print("   Règle : 10 paquets TCP SYN en 10 secondes\n")

target = "scanme.nmap.org"
ports = [21, 22, 23, 25, 53, 80, 110, 143, 443, 3306, 3389, 8080]

print(f"🎯 Cible : {target}")
print(f"📡 Scan de {len(ports)} ports...\n")

start = time.time()

for i, port in enumerate(ports, 1):
    subprocess.run(
        ["docker", "exec", "suricata", "timeout", "0.1", "telnet", target, str(port)],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    
    print(f"  [{i:2d}/12] Port {port:5d} - SYN envoyé")
    time.sleep(0.5)

duration = time.time() - start

print(f"\n✅ {len(ports)} paquets TCP SYN en {duration:.1f} secondes")
print(f"   Seuil : 10 paquets en 10s → {len(ports)} en {duration:.1f}s")
print("\n⚠️  Devrait déclencher : PORT SCAN DETECTED")


🔍 ATTAQUE 1 : PORT SCAN

📋 Objectif : Déclencher 'PORT SCAN DETECTED'
   Règle : 10 paquets TCP SYN en 10 secondes

🎯 Cible : scanme.nmap.org
📡 Scan de 12 ports...

  [ 1/12] Port    21 - SYN envoyé
  [ 2/12] Port    22 - SYN envoyé
  [ 3/12] Port    23 - SYN envoyé
  [ 4/12] Port    25 - SYN envoyé
  [ 5/12] Port    53 - SYN envoyé
  [ 6/12] Port    80 - SYN envoyé
  [ 7/12] Port   110 - SYN envoyé
  [ 8/12] Port   143 - SYN envoyé
  [ 9/12] Port   443 - SYN envoyé
  [10/12] Port  3306 - SYN envoyé
  [11/12] Port  3389 - SYN envoyé
  [12/12] Port  8080 - SYN envoyé

✅ 12 paquets TCP SYN en 7.2 secondes
   Seuil : 10 paquets en 10s → 12 en 7.2s

⚠️  Devrait déclencher : PORT SCAN DETECTED


#### Simulation de burst HTTP en ligne de commande (en ligne de commande ou en python) 

In [81]:
"""
═══════════════════════════════════════════════════════
ATTAQUE - HTTP BURST
═══════════════════════════════════════════════════════
Envoie 20 requêtes HTTP en 1.5 secondes
"""

import subprocess
import time

print("💥 ATTAQUE 2 : HTTP BURST")
print("=" * 60)
print("\n📋 Objectif : Déclencher 'HTTP BURST DETECTED'")
print("   Règle : 15 requêtes HTTP en 2 secondes\n")

target = "http://example.com"
num_requests = 20  # Plus que le seuil de 15

print(f"🎯 Cible : {target}")
print(f"🌊 Envoi de {num_requests} requêtes HTTP...\n")

start = time.time()

for i in range(1, num_requests + 1):
    subprocess.run(
        f'docker exec suricata curl -s {target}',
        shell=True,
        capture_output=True,
        timeout=1
    )
    
    if i % 5 == 0:
        print(f"  [{i:2d}/20] Requêtes envoyées")
    
    time.sleep(0.07)  # 70ms entre chaque = ~14 req/sec

duration = time.time() - start

print(f"\n✅ {num_requests} requêtes HTTP en {duration:.2f} secondes")
print(f"   Taux : {num_requests/duration:.1f} requêtes/seconde")
print(f"   Seuil : 15 requêtes en 2s → {num_requests} en {duration:.2f}s")
print(f"\n⚠️  Devrait déclencher : HTTP BURST DETECTED")

💥 ATTAQUE 2 : HTTP BURST

📋 Objectif : Déclencher 'HTTP BURST DETECTED'
   Règle : 15 requêtes HTTP en 2 secondes

🎯 Cible : http://example.com
🌊 Envoi de 20 requêtes HTTP...

  [ 5/20] Requêtes envoyées
  [10/20] Requêtes envoyées
  [15/20] Requêtes envoyées
  [20/20] Requêtes envoyées

✅ 20 requêtes HTTP en 4.52 secondes
   Taux : 4.4 requêtes/seconde
   Seuil : 15 requêtes en 2s → 20 en 4.52s

⚠️  Devrait déclencher : HTTP BURST DETECTED


#### Simulation de brute force

In [82]:
"""
═══════════════════════════════════════════════════════
ATTAQUE - BRUTE FORCE
═══════════════════════════════════════════════════════
Envoie 8 requêtes avec "pass=" dans l'URI en moins de 10 secondes
"""

import subprocess
import time

print("🔐 ATTAQUE 3 : BRUTE FORCE")
print("=" * 60)
print("\n📋 Objectif : Déclencher 'BRUTE FORCE DETECTED'")
print("   Règle : 6 requêtes avec 'pass=' dans l'URI en 10 secondes\n")

passwords = [
    "admin",
    "password",
    "123456",
    "qwerty",
    "letmein",
    "admin123",
    "root",
    "toor"
]

print(f"🔓 Tentatives de connexion avec {len(passwords)} mots de passe...\n")

start = time.time()

for i, pwd in enumerate(passwords, 1):
    # IMPORTANT : "pass=" doit être dans l'URI
    url = f"http://example.com/login?user=admin&pass={pwd}"
    
    subprocess.run(
        f'docker exec suricata curl -s "{url}"',
        shell=True,
        capture_output=True,
        timeout=2
    )
    
    print(f"  [{i}/8] Tentative avec pass={pwd}")
    time.sleep(0.8)  # 0.8s entre chaque = 8 en ~6.4 secondes

duration = time.time() - start

print(f"\n✅ {len(passwords)} tentatives avec 'pass=' dans l'URI")
print(f"   Durée : {duration:.1f} secondes")
print(f"   Seuil : 6 requêtes en 10s → {len(passwords)} en {duration:.1f}s")
print(f"\n⚠️  Devrait déclencher : BRUTE FORCE DETECTED")

🔐 ATTAQUE 3 : BRUTE FORCE

📋 Objectif : Déclencher 'BRUTE FORCE DETECTED'
   Règle : 6 requêtes avec 'pass=' dans l'URI en 10 secondes

🔓 Tentatives de connexion avec 8 mots de passe...

  [1/8] Tentative avec pass=admin
  [2/8] Tentative avec pass=password
  [3/8] Tentative avec pass=123456
  [4/8] Tentative avec pass=qwerty
  [5/8] Tentative avec pass=letmein
  [6/8] Tentative avec pass=admin123
  [7/8] Tentative avec pass=root
  [8/8] Tentative avec pass=toor

✅ 8 tentatives avec 'pass=' dans l'URI
   Durée : 7.4 secondes
   Seuil : 6 requêtes en 10s → 8 en 7.4s

⚠️  Devrait déclencher : BRUTE FORCE DETECTED


#### SQL injection

In [83]:
import subprocess
import time

print("💉 ATTAQUE 1 : SQL INJECTION")

sql_payloads = [
    "http://example.com/login?user=admin&pass=' OR '1'='1",
    "http://example.com/search?q=' OR '1'='1",
    "http://example.com/page?id=1' OR '1'='1",
    "http://example.com/api?filter=' OR '1'='1 --",
    "http://example.com/data?name=test' OR '1'='1",
]

for i, url in enumerate(sql_payloads, 1):
    subprocess.run(
        f'docker exec suricata curl -s "{url}"',
        shell=True,
        capture_output=True,
        timeout=2
    )
    payload = url.split('=')[-1]
    print(f"[{i}/5] Payload : {payload}")
    time.sleep(0.3)

print("✅ SQL Injection envoyée")

💉 ATTAQUE 1 : SQL INJECTION
[1/5] Payload : '1
[2/5] Payload : '1
[3/5] Payload : '1
[4/5] Payload : '1 --
[5/5] Payload : '1
✅ SQL Injection envoyée


#### XSS injection

In [84]:
import subprocess
import time

print("🎯 ATTAQUE 2 : XSS")

xss_payloads = [
    "http://example.com/search?q=<script>alert('XSS')</script>",
    "http://example.com/comment?text=<script>alert(1)</script>",
    "http://example.com/post?content=<script>document.cookie</script>",
    "http://example.com/input?data=<script>alert('hack')</script>",
]

for i, url in enumerate(xss_payloads, 1):
    subprocess.run(
        f'docker exec suricata curl -s "{url}"',
        shell=True,
        capture_output=True,
        timeout=2
    )
    payload = url.split('=', 1)[-1]
    print(f"[{i}/4] Payload : {payload[:40]}...")
    time.sleep(0.3)

print("✅ XSS envoyée")


🎯 ATTAQUE 2 : XSS
[1/4] Payload : <script>alert('XSS')</script>...
[2/4] Payload : <script>alert(1)</script>...
[3/4] Payload : <script>document.cookie</script>...
[4/4] Payload : <script>alert('hack')</script>...
✅ XSS envoyée


## 4. Détection d'anomalies sans ML

#### Détéctions et clasifications des attaques

L'objectif est de construire des règles qui détectent vos simulations de comportements précédents.

Un exemple de détection d'anomalies basiques basées sur des règles:     

In [91]:
import json
import subprocess
from collections import defaultdict

# Fichier de sortie ou l'on stoker les alertes classifiés
output_file = "detected_attacks.json" 

print("🔍 DÉTECTION ET CLASSIFICATION DES ATTAQUES")
print("=" * 50)

# Charger les logs
result = subprocess.run(
    ["docker", "exec", "suricata", "cat", "/var/log/suricata/eve.json"],
    capture_output=True,
    text=True
)

logs = [json.loads(line) for line in result.stdout.splitlines()]
alerts = [log for log in logs if log.get("event_type") == "alert"]

print(f"\n📊 Total d'alertes : {len(alerts)}\n")

# Classification des attaques
attack_types = {
    "port_scan": [],
    "http_burst": [],
    "brute_force": [],
    "other": []
}

for alert in alerts:
    signature = alert.get("alert", {}).get("signature", "")
    
    if "PORT SCAN" in signature:
        attack_types["port_scan"].append(alert)
    elif "HTTP BURST" in signature:
        attack_types["http_burst"].append(alert)
    elif "BRUTE FORCE" in signature:
        attack_types["brute_force"].append(alert)
    else:
        attack_types["other"].append(alert)

# Afficher les résultats
print("=" * 50)
print("🎯 CLASSIFICATION DES ATTAQUES DÉTECTÉES")
print("=" * 50)

# Port Scan
if attack_types["port_scan"]:
    print(f"\n🔍 SCAN DE PORTS : {len(attack_types['port_scan'])} alertes")
    print("   Caractéristiques :")
    print("   • Multiples connexions TCP SYN vers différents ports")
    print("   • Comportement typique d'un scanner (nmap, masscan)")
    
    # Analyser les détails
    sources = set([a.get("src_ip") for a in attack_types["port_scan"]])
    print(f"   • IPs sources détectées : {len(sources)}")
    for src in list(sources)[:3]:
        print(f"     - {src}")

# HTTP Burst
if attack_types["http_burst"]:
    print(f"\n💥 HTTP BURST (FLOOD) : {len(attack_types['http_burst'])} alertes")
    print("   Caractéristiques :")
    print("   • Volume anormalement élevé de requêtes HTTP")
    print("   • Peut indiquer un DDoS ou un stress test")
    
    # Analyser les détails
    sources = set([a.get("src_ip") for a in attack_types["http_burst"]])
    destinations = set([a.get("dest_ip") for a in attack_types["http_burst"]])
    print(f"   • IPs sources : {len(sources)}")
    print(f"   • Cibles : {len(destinations)}")
    for dst in list(destinations)[:3]:
        print(f"     - {dst}")

# Brute Force
if attack_types["brute_force"]:
    print(f"\n🔐 BRUTE FORCE : {len(attack_types['brute_force'])} alertes")
    print("   Caractéristiques :")
    print("   • Tentatives répétées de connexion")
    print("   • Requêtes vers /login, /admin, /auth")
    print("   • Comportement typique de credential stuffing")
    
    # Analyser les détails
    sources = set([a.get("src_ip") for a in attack_types["brute_force"]])
    print(f"   • IPs sources suspectes : {len(sources)}")
    for src in list(sources)[:3]:
        count = len([a for a in attack_types["brute_force"] if a.get("src_ip") == src])
        print(f"     - {src} : {count} tentatives")

# Autres
if attack_types["other"]:
    print(f"\n📌 AUTRES ALERTES : {len(attack_types['other'])} alertes")

print("\n" + "=" * 50)
print("📋 RÉSUMÉ DES MENACES DÉTECTÉES")
print("=" * 50)

total_threats = sum([
    len(attack_types["port_scan"]),
    len(attack_types["http_burst"]),
    len(attack_types["brute_force"])
])

if total_threats > 0:
    print(f"\n✅ {total_threats} menaces identifiées et classifiées :")
    if attack_types["port_scan"]:
        print(f"   • {len(attack_types['port_scan'])} tentatives de scan de ports")
    if attack_types["http_burst"]:
        print(f"   • {len(attack_types['http_burst'])} attaques par flood HTTP")
    if attack_types["brute_force"]:
        print(f"   • {len(attack_types['brute_force'])} tentatives de brute force")
else:
    print("\n⚠️  Aucune menace spécifique détectée")
    print("   Vérifier que les règles sont bien chargées")
# Fusionner uniquement les attaques identifiées
filtered_attacks = (
    attack_types["port_scan"] +
    attack_types["http_burst"] +
    attack_types["brute_force"]
)

print("=" * 60)
print("🎯 EXPORT DES ALERTES IDENTIFIÉES")
print("=" * 60)
if filtered_attacks:

    # Créer le fichier s’il n’existe pas
    mode = "w"  # écrase l'ancien contenu (tu peux mettre "a" pour append)

    with open(output_file, mode) as f:
        for attack in filtered_attacks:
            f.write(json.dumps(attack) + "\n")

    print(f"\n✅ {len(filtered_attacks)} alertes exportées dans : {output_file}")

    print("\nDétail :")
    print(f"   • Port Scan : {len(attack_types['port_scan'])}")
    print(f"   • HTTP Burst : {len(attack_types['http_burst'])}")
    print(f"   • Brute Force : {len(attack_types['brute_force'])}")

else:
    print("⚠️ Aucune attaque spécifique à exporter.")

print("\n📁 Terminé.")

🔍 DÉTECTION ET CLASSIFICATION DES ATTAQUES

📊 Total d'alertes : 352

🎯 CLASSIFICATION DES ATTAQUES DÉTECTÉES

🔍 SCAN DE PORTS : 2 alertes
   Caractéristiques :
   • Multiples connexions TCP SYN vers différents ports
   • Comportement typique d'un scanner (nmap, masscan)
   • IPs sources détectées : 1
     - 172.18.0.2

💥 HTTP BURST (FLOOD) : 10 alertes
   Caractéristiques :
   • Volume anormalement élevé de requêtes HTTP
   • Peut indiquer un DDoS ou un stress test
   • IPs sources : 3
   • Cibles : 3
     - 104.18.26.120
     - 172.18.0.2
     - 104.18.27.120

🔐 BRUTE FORCE : 1 alertes
   Caractéristiques :
   • Tentatives répétées de connexion
   • Requêtes vers /login, /admin, /auth
   • Comportement typique de credential stuffing
   • IPs sources suspectes : 1
     - 172.18.0.2 : 1 tentatives

📌 AUTRES ALERTES : 339 alertes

📋 RÉSUMÉ DES MENACES DÉTECTÉES

✅ 13 menaces identifiées et classifiées :
   • 2 tentatives de scan de ports
   • 10 attaques par flood HTTP
   • 1 tentatives 

#### Affichage des 5 dernières alertes générés

In [87]:
import json
import subprocess

# Récupère le contenu du fichier eve.json depuis le conteneur
result = subprocess.run(
    ["docker", "exec", "suricata", "cat", "/var/log/suricata/eve.json"],
    capture_output=True,
    text=True
)

# Chaque ligne du fichier est un JSON indépendant
logs = [json.loads(line) for line in result.stdout.splitlines()]

# Filtrer uniquement les alertes
alerts = [log for log in logs if log.get("event_type") == "alert"]

print(f"Total : {len(alerts)} alertes")

# Affiche les 5 dernières alertes
for alert in alerts[-5:]:
    print({
        "timestamp": alert.get("timestamp"),
        "src_ip": alert.get("src_ip"),
        "dest_ip": alert.get("dest_ip"),
        "signature": alert.get("alert", {}).get("signature")
    })

for alert in alerts[-5:]:
    print(alert)


Total : 352 alertes
{'timestamp': '2026-02-15T16:49:46.366865+0000', 'src_ip': '104.18.26.120', 'dest_ip': '172.18.0.2', 'signature': 'HTTP Traffic'}
{'timestamp': '2026-02-15T16:49:46.366894+0000', 'src_ip': '172.18.0.2', 'dest_ip': '104.18.26.120', 'signature': 'HTTP Traffic'}
{'timestamp': '2026-02-15T16:49:46.367058+0000', 'src_ip': '172.18.0.2', 'dest_ip': '104.18.26.120', 'signature': 'HTTP Traffic'}
{'timestamp': '2026-02-15T16:49:46.384037+0000', 'src_ip': '104.18.26.120', 'dest_ip': '172.18.0.2', 'signature': 'HTTP Traffic'}
{'timestamp': '2026-02-15T16:49:46.384063+0000', 'src_ip': '172.18.0.2', 'dest_ip': '104.18.26.120', 'signature': 'HTTP Traffic'}
{'timestamp': '2026-02-15T16:49:46.366865+0000', 'flow_id': 573360068869170, 'in_iface': 'eth0', 'event_type': 'alert', 'src_ip': '104.18.26.120', 'src_port': 80, 'dest_ip': '172.18.0.2', 'dest_port': 39816, 'proto': 'TCP', 'ip_v': 4, 'pkt_src': 'wire/pcap', 'alert': {'action': 'allowed', 'gid': 1, 'signature_id': 1000001, 'rev'

## 5. Détection d'anomalies avec ML

L'objectif est d'à partir de données labelisées comme étant anomarles ou non, construire un jeu de donnée et entrainer un algorithme de machine learning (classifier binaire) sur ces données. Le modèle devra ensuite etre appelé pour prédire les futurs comportements anormaux.

Indice: le modèle Isolation Forest pourrait etre utile

In [ ]:
# TODO

## Bonus
- améliorer la stack (ex: ajout de Wazuh)
- dashboard Kibana pour voir en live les simulations de comportements anormaux et les détection d'anomalie (timeline des alertes, etc.)
- analyse statistique avancée
- simulations de comportement anormaux avancée
- utilisation du module de détection d'anomalie d'Elasticsearch (https://www.elastic.co/docs/explore-analyze/machine-learning/anomaly-detection)
- collaboration en groupe sur le projet Git (Pull requests, commits, etc.)
- utilisation de docker / docker compose / devcontainer
- etc.